In [6]:
from til_environment import gridworld

env = gridworld.env()

In [8]:
from til_environment import gridworld

env = gridworld.env(
    env_wrappers=[],  # clear out default env wrappers
    render_mode="human",  # Render the map; not visible on Workbench
    debug=True,  # Enable debug mode
    novice=True,  # Use same map layout every time (for Novice teams only)
)
env.reset(seed=42)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        break
    else:
        # Insert your policy here
        action = env.action_space(agent).sample()

    env.step(action)

env.close()

In [9]:
import gymnasium as gym
import numpy as np

class GymCompatibleEnv(gym.Env):
    def __init__(self, pettingzoo_env):
        self.env = pettingzoo_env
        self.agents = self.env.possible_agents
        self.agent = self.agents[0]  # Focus on a single agent for simplicity
        self.action_space = self.env.action_space(self.agent)
        self.observation_space = self.env.observation_space(self.agent)

    def reset(self, seed=None, options=None):
        obs = self.env.reset(seed=seed)
        return obs[self.agent], {}

    def step(self, action):
        actions = {agent: self.env.action_space(agent).sample() for agent in self.agents}
        actions[self.agent] = action
        obs, rewards, terminations, truncations, infos = self.env.step(actions)
        return obs[self.agent], rewards[self.agent], terminations[self.agent], truncations[self.agent], infos[self.agent]
